# JupyterLite Workflow

In diesem Repository geht es darum, einen Workflow für JupyterLite zu testen, der es erlaubt, lokal zu entwickeln mit VSCode resp. JupyterLab und anschliessend die Notebooks via JupyterLite im Browser auszuführen.

In einem zweiten Schritt soll der Build-Prozess via GitHub Actions automatisiert werden und die Notebooks anschliessend auf GitHub Pages gehostet werden.

## Standard Python

In [ ]:
print("Hallo World")

## Pandas

In [ ]:
import pandas as pd

df = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6]
})

display(df)

## Requests

In [ ]:
import requests

response = requests.get('https://api.github.com')
print(response.status_code)